# Fase 1.3: Optimización del modelo en HAR (Hailo Archive)

In [1]:
# Imports

import json
import os
import gc

import numpy as np
import tensorflow as tf
from IPython.display import SVG
from matplotlib import patches
from matplotlib import pyplot as plt
from PIL import Image
from tensorflow.python.eager.context import eager_mode
import torch
import onnx
import onnxruntime as ort
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from itertools import islice

# Funciones y parámetros de la CNN base
import sys
sys.path.append('../src/')
import resbagan_networks
import resbagan_datasets

# import the hailo sdk client relevant classes
from hailo_sdk_client import ClientRunner, InferenceContext

2025-06-05 14:12:20.597112: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-05 14:12:21.430768: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/pablo/anaconda3/envs/hailo_gpu_env/lib/python3.8/site-packages/nvidia/dali/backend.py:77: Warning: DALI 1.49 is the last release to support Python 3.8 Please update your environment to use Python 3.9, or newer.
  deprecation_warning(


## 1. Optimización detallada

### 1.1. Carga de datos

In [2]:
# Datos de entrada

batch_size = 100
B = 5
sizex = 32
sizey = 32

In [3]:
# Definir parámetros y cargar datos

DATASET='../data/imagenes_rios/oitaven_river.raw'
GT='../data/imagenes_rios/oitaven_river.pgm'
MODEL="../results/models/model_ResBaGAN.pt"

# Queremos usar todos los datos para la inferencia
SAMPLES=[0.0,0.0]

# Carga de datos para la inferencia en el discriminador
dataset = resbagan_datasets.HyperDataset(
    "oitaven_river", segmented=False, patch_size=32, ratios=(SAMPLES[0], SAMPLES[1]))

# Almacenamos las dimensiones en variables
H = dataset.height
V = dataset.width

samples = dataset.ordered_test_set['samples']

# Obtenemos el array de gt
truth = dataset.gt.flatten()
# Obtenemos un array de indices para test
test = dataset.test_index_list

# Obtenemos el numero de clases y el numero de clases no vacias para test
nclases = dataset.classes_count
nclases_no_vacias = 0
for i in range(nclases):
    clase_actual = i + 1
    if any(truth[idx] == clase_actual for idx in test):
        nclases_no_vacias += 1

print(dataset)

# The custom HyperDataset object contains all the train, validation and test data
#   --> But it will wrapped into a PyTorch data feeder for convenience
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=100,
    shuffle=False,
    num_workers=4,
    pin_memory=False,
)

[*] Loading dataset oitaven_river from disk
	oitaven_river dataset is in RAW format
[*] Recording available classes
[*] Starting preprocessing
[*] Scaling dataset to [-1, 1]
[*] Splitting dataset into train, validation, and test sets: ratios (0.0, 0.0)
[*] Total samples: 6067179
	[*] Recording samples for class Water (309248 items)
	[*] Recording samples for class Bare soil (113324 items)
	[*] Recording samples for class Rock (79152 items)
	[*] Recording samples for class Asphalt (43861 items)
	[*] Recording samples for class Concrete (128022 items)
	[*] Recording samples for class Tiles (78785 items)
	[*] Recording samples for class Meadows (2428482 items)
	[*] Recording samples for class Native trees (1829360 items)
	[*] Recording samples for class Pines (193884 items)
	[*] Recording samples for class Eucalyptus (863061 items)

[*] HyperDataset summary:
	Name: oitaven_river
	Shape: (height) 6689, (width) 6722, (bands) 5
	Classes: ['Water', 'Bare soil', 'Rock', 'Asphalt', 'Concrete', 

In [4]:
# Cargar el modelo convertido a ONNX

ort_session = ort.InferenceSession("../results/models/model_ResBaGAN_discriminator.onnx")

# Cargar el modelo en HAR

model_name = "../results/models/model_ResBaGAN_discriminator"
hailo_model_har_name = f"{model_name}_hailo_model.har"
assert os.path.isfile(hailo_model_har_name), "Please provide valid path for HAR file"
runner = ClientRunner(har=hailo_model_har_name, hw_arch="hailo8l")

### 1.2 Evaluar el modelo en har sin optimización

In [12]:
# Comprobacion rapida de la diferencia de precisión

# Crear tensor de entrada de prueba
input_tensor = (torch.rand(1, B, sizex, sizey) * 2 - 1).to("cpu")
input_np = input_tensor.cpu().numpy()

# Realizar la inferencia en ONNX
output_onnx = ort_session.run(None, {'input': input_np})[0]

print(output_onnx)

# Realizar la inferencia en HAR
with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
    input_har = np.transpose(input_np, (0, 2, 3, 1))

    # Realizar la inferencia en el modelo .har
    output_har = runner.infer(ctx, input_har)[0]

    print(output_har) 

# Comparar las diferencias
error = np.abs(output_onnx - output_har).mean()
print(f'Error medio entre ONNX y HAR: {error}')

[[ -3.8289707    1.7461531    0.17802365 -10.280189    -4.26982
    2.2362123   -6.9854684   -4.3605914    6.738623    -5.441714
    9.253869  ]]
[info] Using 1 GPU for inference


2025-06-05 14:14:23.022559: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-06-05 14:14:23.130953: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 8entries [00:00, 253.99entries/s]

[[[[-2.8285937   1.4508626   0.37640417 -9.220876   -1.9980316
     0.79922646 -3.0193255  -3.1164143   3.4191551  -3.9640741
     6.231791  ]]]]
Error medio entre ONNX y HAR: 1.7537856101989746


In [7]:
# Evaluar el modelo en HAR sin optimizacion

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    inputs, labels = next(iter(test_loader))
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

Inference: 0entries [00:00, ?entries/s]2025-04-18 03:17:18.366492: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1540.91entries/s]


[Iteración 1] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:17:18.861796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2050.92entries/s]


[Iteración 2] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:17:19.306366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2172.28entries/s]


...

[Iteración 98] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:02.892880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2248.20entries/s]


[Iteración 99] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:03.326304: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2316.19entries/s]

[Iteración 100] Error medio: 0.0000 ; Coincidencias: 100/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.000000
Precisión global: 100.00%


### 1.3 Aplicar modificaciones de optimización al modelo y evaluarlo

In [10]:
# Crear un model script para el proceso de optimización

model_script_lines = [
    # Add normalization layer with mean [123.675, 116.28, 103.53] and std [58.395, 57.12, 57.375])
    # "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n",
    # For multiple input nodes:
    # {normalization_layer_name_1} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_1_from_hn})\n',
    # {normalization_layer_name_2} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_2_from_hn})\n',
    # ...
]

# Load the model script to ClientRunner so it will be considered on optimization
runner.load_model_script("".join(model_script_lines))
runner.optimize_full_precision()

[warning] Model script is empty
[info] Loading model script commands to model_cnn21 from string
[warning] Model script is empty
[warning] DEPRECATION WARNING: Optimizing in full precision will require calibration data in the near future, to allow more accurate optimization algorithms which require inference on actual data.


In [9]:
# Evaluar el modelo en HAR con optimizacion

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    inputs, labels = next(iter(test_loader))
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_FP_OPTIMIZED) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:34.587670: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 270.39entries/s]


[Iteración 1] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:35.459731: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2039.36entries/s]


[Iteración 2] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:35.912833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2168.35entries/s]


...

[Iteración 98] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:19:19.253537: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2241.04entries/s]


[Iteración 99] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:19:19.710452: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2302.29entries/s]

[Iteración 100] Error medio: 0.0000 ; Coincidencias: 100/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.000000
Precisión global: 100.00%


### 1.4 Cuantizar el modelo y evaluarlo

In [5]:
# Crear un dataset de calibración
# The original images are being used, just as the input to the SDK_FP_OPTIMIZED emulator
total_images = 1050

calib_dataset = np.zeros((total_images, sizex, sizey, B), dtype = np.float32)

count = 0

# Seleccionamos batches del dataloader para guardarlos en el dataset
for inputs, _ in test_loader:
    for img in inputs:
        if count >= total_images:
            break
            
        # Los inputs son de la forma (batch_size, B, sizex, sizey)
        img_np = img.numpy()
        # Trasponemos los inputs a formato (batch_size, sizex, sizey, B)
        img_har = np.transpose(img_np, (1, 2, 0))

        calib_dataset[count] = img_har
        count += 1
        
    if count >= total_images:
        break

In [6]:
# Cuantizar el modelo con el dataset de calibración

# For calling Optimize, use the short version: runner.optimize(calib_dataset)
# A more general approach is being used here that works also with multiple input nodes.
# The calibration dataset could also be a dictionary with the format:
# {input_layer_name_1_from_hn: layer_1_calib_dataset, input_layer_name_2_from_hn: layer_2_calib_dataset}
hn_layers = runner.get_hn_dict()["layers"]
print("Input layers are: ")
print([layer for layer in hn_layers if hn_layers[layer]["type"] == "input_layer"])  # See available input layer names
calib_dataset_dict = {"model_cnn21/input_layer1": calib_dataset}  # In our case there is only one input layer

optimization_level = -100
compression_level = 0
# Mapeamos las proporciones de pesos de 4 bits según el nivel de compresión
compression_ratios = {
    0: 0.0,
    1: 0.2,
    2: 0.4,
    3: 0.6,
    4: 0.8,
    5: 1.0
}
auto_4bit_ratio = compression_ratios.get(compression_level, 0.0)

alls_lines = [
    # "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n",
    # Batch size is 8 by default; 2 was used for stability on PCs with low amount of RAM / VRAM
    f"model_optimization_flavor(optimization_level={optimization_level}, compression_level={compression_level}, batch_size=8)\n",
    # The following line is needed because this is a really small model, and the compression_level is always reverted back to 0.'
    # To force using compression_level with small models, the following line should be used (compression level=4 equals to 80% 4-bit):
    f"model_optimization_config(compression_params, auto_4bit_weights_ratio={auto_4bit_ratio})\n",
    # The application of the compression could be seen by the [info] messages: "Assigning 4bit weight to layer .."
]

runner.load_model_script("".join(alls_lines))

runner.optimize(calib_dataset_dict)

Input layers are: 
['model_cnn21/input_layer1']
[info] Loading model script commands to model_cnn21 from string
[info] Starting Model Optimization


2025-05-27 12:28:33.889242: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-27 12:28:33.894902: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-27 12:28:33.895130: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[warning] Running model optimization with zero level of optimization is not recommended for production use and might lead to suboptimal accuracy results
[info] Model received quantization params from the hn


2025-05-27 12:28:37.967115: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-27 12:28:37.968883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-27 12:28:37.969107: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[info] MatmulDecompose skipped
[info] Starting Mixed Precision
[info] Model Optimization Algorithm Mixed Precision is done (completion time is 00:00:00.02)
[info] LayerNorm Decomposition skipped
[info] Starting Statistics Collector
[info] Using dataset with 64 entries for calibration


...

[info] Model Optimization Algorithm Statistics Collector is done (completion time is 00:00:04.64)
[info] Starting Fix zp_comp Encoding
[info] Model Optimization Algorithm Fix zp_comp Encoding is done (completion time is 00:00:00.00)
[info] Matmul Equalization skipped
[info] Starting MatmulDecomposeFix
[info] Model Optimization Algorithm MatmulDecomposeFix is done (completion time is 00:00:00.00)


2025-05-27 12:28:46.907044: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-05-27 12:28:46.915181: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [8,32,32,5]
	 [[{{node Placeholder/_0}}]]
2025-05-27 12:28:46.922483: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [8,32,32,5]
	 [[{{node Placeholder/_0}}]]
2025-05-27 12:28:46.928020: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor star

[info] Finetune encoding skipped
[info] Bias Correction skipped
[info] Adaround skipped
[info] Quantization-Aware Fine-Tuning skipped


2025-05-27 12:28:53.231189: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-27 12:28:53.233219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-27 12:28:53.233445: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[info] Layer Noise Analysis skipped
[info] Model Optimization is done


In [7]:
# Evaluar el modelo en HAR cuantizado

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    inputs, labels = next(iter(test_loader))
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_QUANTIZED) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

2025-05-27 12:29:25.737724: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-27 12:29:25.738903: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-27 12:29:25.739104: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[info] Using 1 GPU for inference


2025-05-27 12:29:27.169127: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-05-27 12:29:28.156444: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-05-27 12:29:32.474111: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2025-05-27 12:29:33.048232: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Inference: 104entries [00:05, 20.71entries/s]


[Iteración 1] Error medio: 0.0500 ; Coincidencias: 97/100
[info] Using 1 GPU for inference


2025-05-27 12:29:33.548364: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-05-27 12:29:33.623320: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1981.51entries/s]


...

[Iteración 98] Error medio: 0.0500 ; Coincidencias: 96/100
[info] Using 1 GPU for inference


2025-05-27 12:30:23.419153: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-05-27 12:30:23.502419: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1841.00entries/s]


[Iteración 99] Error medio: 0.1500 ; Coincidencias: 94/100
[info] Using 1 GPU for inference


2025-05-27 12:30:23.975171: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-05-27 12:30:24.060861: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1905.28entries/s]

[Iteración 100] Error medio: 0.0300 ; Coincidencias: 97/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.091000
Precisión global: 95.60%


In [8]:
# Guardar el modelo cuantizado
# Let's save the runner's state to a Quantized HAR
quantized_model_har_path = f"{model_name}_quantized_model_o{optimization_level}_c{compression_level}.har"
runner.save_har(quantized_model_har_path)

[info] Saved HAR to: /home/pablo/Documentos/Enxeñaría Informática/Cuarto/2º Cuatrimestre/Traballo Fin de Grao/hyper-rpi/results/model_cnn21_quantized_model_o-100_c0.har


In [50]:
# Verifica el input
print("input_np.shape:", input_np.shape)              # Esperado: (1, 5, 32, 32)
print("input_har.shape:", input_har.shape)            # Esperado: (1, 32, 32, 5)
print("input_np mean/std:", input_np.mean(), input_np.std())
print("input_har mean/std:", input_har.mean(), input_har.std())


input_np.shape: (1, 5, 32, 32)
input_har.shape: (1, 32, 32, 5)
input_np mean/std: 0.0012923591 0.57947564
input_har mean/std: 0.0012923591 0.57947564


SyntaxError: invalid character in identifier (1971112835.py, line 1)

In [44]:
print(runner.get_hn())


OrderedDict([('name', 'model_ResBaGAN_discriminator'), ('net_params', OrderedDict([('version', '1.0'), ('stage', 'HN'), ('clusters_placement', [[]]), ('clusters_to_skip', []), ('output_layers_order', ['model_ResBaGAN_discriminator/fc1', 'model_ResBaGAN_discriminator/avgpool1']), ('is_transformer', False), ('transposed_net', False), ('net_scopes', ['model_ResBaGAN_discriminator']), ('lora_adapters', [])])), ('layers', OrderedDict([('model_ResBaGAN_discriminator/input_layer1', OrderedDict([('type', 'input_layer'), ('input', []), ('output', ['model_ResBaGAN_discriminator/conv1']), ('input_shapes', [[-1, 32, 32, 5]]), ('output_shapes', [[-1, 32, 32, 5]]), ('original_names', ['input']), ('compilation_params', {}), ('quantization_params', {}), ('transposed', False), ('engine', 'nn_core'), ('io_type', 'standard')])), ('model_ResBaGAN_discriminator/conv1', OrderedDict([('type', 'conv'), ('input', ['model_ResBaGAN_discriminator/input_layer1']), ('output', ['model_ResBaGAN_discriminator/external

In [40]:
output_har = output_har.reshape(output_onnx.shape)


In [49]:
output_har_reshaped = output_har.reshape(output_onnx.shape)
diff = np.abs(output_onnx - output_har_reshaped)
print("Error medio:", diff.mean())
print("Error máximo:", diff.max())


Error medio: 2.24464
Error máximo: 5.786163


In [42]:
error = np.abs(output_onnx - output_har).mean()
print(f"Error medio entre ONNX y HAR: {error:.6f}")


Error medio entre ONNX y HAR: 16.051535
